### 테이블 및 데이터 초기화(전부삭제)

In [21]:
"""
이 코드를 실행하면 DB에 저장된 모든 테이블 삭제 
data 폴더에 저장된 모든 이미지파일 삭제

"""

import sqlite3
import os
import shutil

conn = sqlite3.connect('database.db')
c = conn.cursor()
sql = """
DROP TABLE IF EXISTS users;
CREATE TABLE users (
           id integer unique primary key autoincrement,
           name text,
           gender text
);
"""
c.executescript(sql) # 한번의 호출로 여러 SQL문을 실행할때 사용

if os.path.exists("./dataset"):
    shutil.rmtree("./dataset")
else:
    os.mkdir("./dataset")

conn.commit()
conn.close()

### 선택한 사용자만 삭제

In [28]:
"""
이 코드를 실행하면 선택된 회원의 DB 삭제 
data 폴더에 저장된 선택된 회원의 모든 이미지파일 삭제

"""

import sqlite3
import os
import shutil
import glob

conn = sqlite3.connect('database.db')
c = conn.cursor()

uname = input("Enter your name: ")


c.execute("SELECT * FROM users WHERE name=(?)",(uname,))
result = c.fetchone()

try:
    if uname in result:
        c.execute("DELETE FROM users WHERE name=(?)",(uname,)) 
        os.system('rm ' + './dataset/' + uname + '*' + '.jpg')
        print("Successfully deleted")
except TypeError: 
    print("not exists, please check your id")


conn.commit()
conn.close()

Enter your name: kim
(3, 'kim', 'male')
Successfully deleted


### 사진촬영 후 DB에 저장 (사용자등록)

In [29]:
"""
추가 사용자 등록을 할 경우 여기서부터 실행

"""

import sqlite3
import cv2
import numpy as np 
import sqlite3
import os
from PIL import Image
import glob
from keras.models import load_model


conn = sqlite3.connect('database.db')
if not os.path.exists('./dataset'):
    os.makedirs('./dataset')
    
# Connection를 얻으면 cursor() 객체를 만들어야 excute()메서드 호출가능
c = conn.cursor()
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
uname = input("Enter your name: ")

c.execute('INSERT INTO users (name) VALUES (?)', (uname,))

# 읽기전용 어트리뷰트로 마지막으로 수정된 행의 rowid() 제공
# excute() 메서드를 사용하여 insert나 replace를 실행했을때만 설정
uid = c.lastrowid
print("\n [NOTICE] Initializing face capture. Look the camera and wait a few moment")
count = 0
while True:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        count = count+1
        cv2.imwrite("dataset/"+str(uname)+"."+str(uid)+"."+str(count)+".jpg",gray[y:y+h,x:x+w])
        cv2.rectangle(img, (x,y), (x+w, y+h), (255,255,255), 2)
        cv2.waitKey(100)
    cv2.imshow('img',img)
    if cv2.waitKey(1)==13 or count == 50:
        break
print("\n [NOTICE] Exiting camera and cleanup stuff")
cap.release()
cv2.destroyAllWindows()


caltech_dir = './dataset'

image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
filenames = []

files = glob.glob(caltech_dir+"/"+uname+"*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)
X = X.astype(float) / 255
model = load_model('./model/male_female_classify.model')#학습시킨 모델

prediction = model.predict(X)

cnt = 0
cnt_m = 0
cnt_f = 0
for i in prediction:
    if i >= 0.5: #여자일때
        cnt_f +=1
    else:        #남자일때
        cnt_m +=1
    cnt += 1

if cnt_m > cnt_f:
    sql = "UPDATE users SET gender=(?) WHERE name=(?)"
    c.execute(sql,("male",uname)) 
    print(f"\n [NOTICE] {uname} is male")
else:
    sql = "UPDATE users SET gender=(?) WHERE name=(?)"
    c.execute(sql,("female",uname))
    print(f"\n [NOTICE] {uname} is female")
    
print("\n [NOTICE] be close the registration program")

conn.commit()
conn.close()

Enter your name: kim

 [NOTICE] Initializing face capture. Look the camera and wait a few moment

 [NOTICE] Exiting camera and cleanup stuff

 [NOTICE] kim is male

 [NOTICE] be close the registration program


### 저장한 이미지를 학습후 데이터 직렬화로 저장

In [23]:
"""
사용자 등록은 여기까지 실행시켜줘야 하나의 데이터로 묶인다

"""

import os
import cv2
import numpy as np 
from PIL import Image

recognizer = cv2.face.LBPHFaceRecognizer_create()
path = 'dataset'

if not os.path.exists('./recognizer'):
    os.makedirs('./recognizer')
def getImagesWithID(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]
    faces = []
    IDs = []
    for imagePath in imagePaths:
        faceImg = Image.open(imagePath).convert('L')
        faceNp = np.array(faceImg,'uint8')
        ID = int(os.path.split(imagePath)[-1].split('.')[1])
        faces.append(faceNp)
        IDs.append(ID)
        cv2.imshow("training",faceNp)
        cv2.waitKey(10)
    return np.array(IDs), faces
Ids, faces = getImagesWithID(path)
recognizer.train(faces,Ids)
recognizer.save('recognizer/trainingData.yml')
cv2.destroyAllWindows()

### 사용자 인증 및 성별 확인

In [1]:
"""
상시 실행 후 얼굴 분석 후 결과값 도출
나온 결과값으로 성별 확인

"""

#### 사용자 인증 ####

import cv2
import numpy as np 
import sqlite3
import os

from PIL import Image
import glob
from keras.models import load_model

conn = sqlite3.connect('database.db')
c = conn.cursor()
fname = "recognizer/trainingData.yml"
if not os.path.isfile(fname):
    print("Please train the data first")
    exit(0)
    
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read(fname)

while True:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,128,0),3)
        ids,conf = recognizer.predict(gray[y:y+h,x:x+w])
        c.execute("SELECT name FROM users WHERE id = (?);", (ids,))
        
        # 쿼리문 질의 결과의 모든 행을 가져와서 리스트를 반환, 행이 없으면 빈 리스트가 반환
        result = c.fetchall()
        name = result[0][0]
        if conf < 50:
            cv2.putText(img, name, (x+3,y+h-3), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,0),2)
        else:
            cv2.putText(img, 'No Match', (x+3,y+h-3), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,220),2)
    cv2.imshow('DTWORESOURCE',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27: # esc 키 누르면 종료
        break
        
print(f" [NOTICE] {name.upper()}, Identify confirmation")
print(f"\n [NOTICE] Welcome {name.upper()}") 
cap.release()
cv2.destroyAllWindows()

#### 성별 확인 ####

c.execute("SELECT gender FROM users WHERE name=(?)",(name,))
# 쿼리문 질의 결과의 단일 행을 가져와서 리스트를 반환
gen = c.fetchone()

if "male" in gen:
    gen = "male"
elif "female" in gen:
    gen = "female"
    
print(f"\n [NOTICE] {name}'s gender is {gen}")

Using TensorFlow backend.


 [NOTICE] KYH, Identify confirmation

 [NOTICE] Welcome KYH

 [NOTICE] kyh's gender is male


### flask 이용하여 결과값 웹에서 얻기

In [ ]:
from flask import Flask, request, render_template, url_for

# Initialize
app = Flask(__name__)
 
# Define a route for url 
@app.route('/')
def form():
    return render_template('face.html')

# form action
@app.route('/hello',methods=['GET'])
def action():
    return render_template('face.html', name = name.upper())

# Run the app 
if __name__ == '__main__':
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Sep/2019 19:24:10] "GET /hello HTTP/1.1" 200 -


### socket 이용하여 server에서 결과값 얻기

In [2]:
from socket import *
from select import *
import sys
from time import ctime

HOST = '127.0.0.1'
PORT = 6000
BUFSIZE = 1024
ADDR = (HOST,PORT)

clientSocket = socket(AF_INET, SOCK_STREAM)# 서버에 접속하기 위한 소켓을 생성한다.

try:
    clientSocket.connect(ADDR)# 서버에 접속을 시도한다.
    clientSocket.send(name.encode())	# 서버에 메시지 전달

except  Exception as e:
    print('%s:%s'%ADDR)
    sys.exit()

print('connect is success')

connect is success


### TTS를 이용하여 결과값 음성으로 듣기

In [16]:
from gtts import gTTS
text = "hi! how are you"+name

tts = gTTS(text=text, lang='en')
tts.save("helloEN.mp3")

### 회원 이름 수정

In [41]:
"""
이 코드를 실행하면 선택된 회원의 DB 이름 변경
data 폴더에 저장된 선택된 회원의 모든 이미지파일 이름 변경

"""

import sqlite3
import os
import shutil


conn = sqlite3.connect('database.db')
c = conn.cursor()

uname = input("Enter your name: ")
sname = input("Enter your nane to change: ")


c.execute("SELECT * FROM users WHERE name=(?)",(uname,))
result = c.fetchone()
            
try:
    if uname in result:
        sql = "UPDATE users SET name=(?) WHERE name=(?)"
        c.execute(sql,(sname,uname)) 
        
        print("Successfully changed")
        
        # 현재 위치(./dataset)의 파일을 모두 가져온다. 
        path = "./dataset"
        for filename in os.listdir(path): 
        
            # 파일 확장자가 (jpg)인 것만 처리 
            if filename.endswith("jpg"): 
            
                # 파일명에서 uname를 sname로 변경하고 파일명 수정 
                new_filename = filename.replace(uname, sname) 
                os.rename(path+"/"+filename, path+"/"+new_filename)
except TypeError: 
    print("not exists, please check your id")

conn.commit()
conn.close()

Enter your name: hoho
Enter your nane to change: kim
Successfully changed


In [ ]:
caltech_dir = './dataset'

image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
filenames = []

files = glob.glob(caltech_dir+"/"+name+"*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)
X = X.astype(float) / 255
model = load_model('./model/male_female_classify.model')#학습시킨 모델

prediction = model.predict(X)

cnt = 0
cnt_m = 0
cnt_f = 0
for i in prediction:
    if i >= 0.5: #여자일때
        cnt_f +=1
    else:        #남자일때
        cnt_m +=1
    cnt += 1

print(f"\n{name} 사용자는 남자입니다" if cnt_m > cnt_f else f"{name} 사용자는 여자입니다")